In [ ]:
import sys
import json
import pandas as pd
from uuid import uuid4

import ipywidgets as widgets
from IPython.display import display
from IPython.core.display import Markdown

import rafal

# Config loading : modify in the config.json file
config = json.load(open('config.json'))
user= config['Rafal.user']
url = config['Rafal.url']

pwd_exists = bool(config.get('Rafal.password', None)) 
wPwd= widgets.Password(value= '',
                      placeholder= 'from config' if pwd_exists else 'Enter password',
                      disabled= pwd_exists
                     )

display(widgets.HBox([widgets.Label('Password for Rafal API :'), wPwd]))

# proxy parameters
proxies = (config['proxies'] if config['proxies']['http'] or config['proxies']['https'] 
           else None)

In [ ]:
# get password
pwd= config['Rafal.password'] if pwd_exists else wPwd.value
if not pwd:
    print('Please write your password first !')

# open a new rafal session and connect
session= rafal.Session(verbose= True)
session.connect(url= url, login= user, pwd= pwd, proxies= proxies)

# Python version
print (f'Python version: {sys.version}')
print (f'Pandas version: {pd.__version__}')

swagger_link= Markdown(f"[Swagger on {session.url}]({session.url}/docs/swagger-ui/index.html?url=/assets/swagger.json#/)")

# list all modules
result= session.request(endPoint= '/modules', http= 'GET', verbose= True)
print('Modules :\n', result)

In [ ]:
swagger_link

## Monitoring Module

### using monitoring_flat

In [ ]:
module= 'monitoring_flat'

### get pivots in this module

In [ ]:
# get all pivots and buil a dict of pivots
res= session.request(endPoint= f'/module/{module}/pivot', http= 'GET')
pivots= {pivot['name']: pivot for pivot in res}


In [ ]:
pivots

In [ ]:
# pivot names
pivots.keys()

### Create a pivot and send a request from it 

In [ ]:
pivots['endpoint_database_queries']

In [ ]:
p= rafal.Pivot(pivots['endpoint_database_queries'])

In [ ]:
type(p)

In [ ]:
p.metrics

In [ ]:
p.dimensions

In [ ]:
# list all metrics
p.metrics

In [ ]:
# list all metrics
p.metrics.keys()

In [ ]:
f1= rafal.Filter(dimension= "request_path", match= '^/[^dlp]')
f2= rafal.Filter(dimension= "username", values= 'guillaume')
f1

In [ ]:
f2

In [ ]:
f1 & f2

In [ ]:
fields= ["access_time", "result_code", "request_id", "request_path", "username"]

In [ ]:
df= session.reqPivot(pivot= p, 
                     metrics= ['count'], 
                     by= fields, 
                     where= f1 & f2
                    )
df1= df.sort_values('access_time')

print(f"{len(df1)} requests recorded from {df1.access_time.iloc[0]} to {df1.access_time.iloc[-1]} (UTF Timezone)")

In [ ]:
df1

In [ ]:
df1.request_path.unique()

In [ ]:
version= session.branches(table= p.table, user= session.user, output= 'versioning')
version

In [ ]:
reqdef= p.reqdef(metrics= ['count'], 
         by= fields, 
         where= (f1 & f2).content, 
         versioning= version)
reqdef

In [ ]:
df1= session.request(endPoint=f'/module/{p.module}/query', jsonText= reqdef, output= 'json')
df1

In [ ]:
# check last request body
session.last_body()